# Threshold based black hole segmentation

This code was used to create T1 lesion (black hole) masks through a thresholding technique. The tissue segmentations performed by the FSL algorithm FAST was used to extract grey matter segmentations. Subsequently the 5th percentile of the grey matter intensities was calculated, creating a threshold value. All voxels in the T2 lesions that had an intensity lower than this threshold, was classified as a black hole voxel.

In [ ]:
# Imports
import nibabel as nib
import numpy as np
import seaborn as sns
import os

In [ ]:
# Path to T2 lesion masks
lesion_masks_path = f"/output_inference/"

# Path to save T1 lesion masks
output_path = "/T1_lesions/"

# Path to T1-weighted volumes
t1_img_path = f"/input_inference/"

# List of subjects
visits = sorted(os.listdir(lesion_masks_path))

# Path to FSL Anat folders
anat_path = f"/fsl_preprocessed/"


for patient_visit in visits:

    # Load lesion mask and T1 volume 
    patient = patient_visit.split("_")[0]  
    lesion_mask_full_path = os.path.join(lesion_masks_path, patient_visit, f"MS_{patient}.nii.gz")
    lesion_mask = nib.load(lesion_mask_full_path)

    t1_img_full_path = os.path.join(t1_img_path, patient_visit, f"MS_{patient}_0001.nii.gz")
    t1_img = nib.load(t1_img_full_path).get_fdata()

    anat_imgs = os.path.join(anat_path, patient_visit)

    # Get tissue segmentations
    for folder in os.listdir(anat_imgs):
        if "t1" in folder:
            all_segs_path = os.path.join(anat_imgs, folder, 'T1_fast_pveseg.nii.gz')

    # Extract grey matter segmentation
    all_segs = nib.load(all_segs_path).get_fdata()
    gm_dist_t1 = t1_img[all_segs==2]

    # Generate black hole mask based on 5th percentile of grey matter
    black_hole_mask = (lesion_mask.get_fdata() == 1) & (t1_img < np.percentile(gm_dist_t1, 5))
    new_mask = nib.Nifti1Image(black_hole_mask, lesion_mask.affine, lesion_mask.header)

    # Save generated mask
    output = os.path.join(output_path, f"{patient_visit}.nii.gz")
    #nib.save(new_mask, output)
    print(f"{patient_visit} done")

print("ALL DONE")